<h1>Φόρτωση βιβλιοθηκών και dataset</h1>

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import keras
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Input # type: ignore
from tensorflow.keras.metrics import MeanSquaredError # type: ignore
import pandas as pd
import matplotlib.pyplot as plt

# Read dataset 
dataset = pd.read_csv("alzheimers_disease_data.csv")


<h1>Προεπεξεργασία Δεδομένων</h1>

In [8]:
# Αφαίρεση μη χρήσιμων στηλών
dataset = dataset.drop(columns=["PatientID", "DoctorInCharge"])

# Split into input and output
X = dataset.drop(columns=["Diagnosis"])
Y = dataset["Diagnosis"]

# Features normalization
X = StandardScaler().fit_transform(X=X)


<h1>Παράμετροι Εκπαίδευσης</h1>

In [4]:
I = X.shape[1]  # Number of columns
H = [
    I//2, 
    2*I//3, 
    I,
    2*I
          ]  # Διαφορετικά μεγέθη κρυφών νευρώνων

h = [0.001, 0.001, 0.05, 0.01]  # Βήμα εκμάθησης
m = [0.2, 0.6, 0.6, 0.6] # Σταθερά momentum

# Split the data to training and testing data 5-Fold
kfold = KFold(n_splits=5, shuffle=True)

epochs = 32  # Αριθμός εποχών
batch_size = 32  # Μέγεθος δέσμης

In [ ]:
I = X.shape[1]  # Number of columns
H = [
    I//2, 
    2*I//3, 
    I,
    2*I
          ]  # Διαφορετικά μεγέθη κρυφών νευρώνων

h = [0.001, 0.001, 0.05, 0.01]  # Βήμα εκμάθησης
m = [0.2, 0.6, 0.6, 0.6] # Σταθερά momentum

# Split the data to training and testing data 5-Fold
kfold = KFold(n_splits=5, shuffle=True)

epochs = 32  # Αριθμός εποχών
batch_size = 32  # Μέγεθος δέσμης

<h1>Εκπαίδευση και Προβολή Αποτελεσμάτων</h1>
<h2>Αλλαγή στον αριθμό των νευρώνων</h2>

In [ ]:
for i in range(len(H)+1):
    print("Number of neurons in the hidden layer: ", H[i])

    # Αποθήκευση τιμών για το διάγραμμα
    avg_loss_per_epoch = np.zeros(epochs)
    avg_acc_per_epoch = np.zeros(epochs)

    for j, (train, test) in enumerate(kfold.split(X)):
        lossList = []
        mseList = []
        accuracyList = []

        # Δημιουργία μοντέλου
        model = Sequential()
        model.add(Input(shape=(I,)))
        model.add(Dense(H[i], activation="relu"))
        model.add(Dense(1, activation="sigmoid"))

        # Compile model
        optimizer = keras.optimizers.SGD(learning_rate=h[0], momentum=m[0])
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[MeanSquaredError(), 'accuracy'])

        # Εκπαίδευση μοντέλου και αποθήκευση ιστορικού
        history = model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0)

        # Αποθήκευση της απώλειας και της ακρίβειας ανά εποχή
        avg_loss_per_epoch += np.array(history.history['loss']) / kfold.get_n_splits()
        avg_acc_per_epoch += np.array(history.history['accuracy']) / kfold.get_n_splits()

        # Αξιολόγηση μοντέλου
        scores = model.evaluate(X[test], Y[test], verbose=0)
        lossList.append(scores[0])
        mseList.append(scores[1])
        accuracyList.append(scores[2])
        print(f"Fold {j}: Loss={scores[0]:.4f}, MSE={scores[1]:.4f}, Accuracy={scores[2]:.4f}")

    print(f"Average Loss: {np.mean(lossList):.4f}")
    print(f"Average MSE: {np.mean(mseList):.4f}")
    print(f"Average Accuracy: {np.mean(accuracyList):.4f}\n")
        # Σχεδίαση των γραφημάτων σύγκλισης
    plt.figure(figsize=(12, 5))

    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), avg_loss_per_epoch, marker='o', linestyle='-', color='r')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"Loss per Epoch (Hidden Neurons: {H[i]})")
    plt.grid()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), avg_acc_per_epoch, marker='o', linestyle='-', color='b')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy per Epoch (Hidden Neurons: {H[i]})")
    plt.grid()

    plt.show()


<h2>Αλλαγή στις παραμέτρους εκμάθησης <b>h</b> και <b>m</b></h2>

In [ ]:
for i in range(len(h)):
    print(f"Learning Rate: {h[i]}")
    print(f"Momentum: {m[i]}")

    # Αποθήκευση τιμών για το διάγραμμα
    avg_loss_per_epoch = np.zeros(epochs)
    avg_acc_per_epoch = np.zeros(epochs)

    for j, (train, test) in enumerate(kfold.split(X)):
        lossList = []
        mseList = []
        accuracyList = []

        # Δημιουργία μοντέλου
        model = Sequential()
        model.add(Input(shape=(I,)))
        model.add(Dense(H[0], activation="relu"))
        model.add(Dense(1, activation="sigmoid"))

        # Compile model
        optimizer = keras.optimizers.SGD(learning_rate=h[0], momentum=m[0])
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[MeanSquaredError(), 'accuracy'])

        # Εκπαίδευση μοντέλου και αποθήκευση ιστορικού
        history = model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0)

        # Αποθήκευση της απώλειας και της ακρίβειας ανά εποχή
        avg_loss_per_epoch += np.array(history.history['loss']) / kfold.get_n_splits()
        avg_acc_per_epoch += np.array(history.history['accuracy']) / kfold.get_n_splits()

        # Αξιολόγηση μοντέλου
        scores = model.evaluate(X[test], Y[test], verbose=0)
        lossList.append(scores[0])
        mseList.append(scores[1])
        accuracyList.append(scores[2])
        print(f"Fold {j}: Loss={scores[0]:.4f}, MSE={scores[1]:.4f}, Accuracy={scores[2]:.4f}")

    print(f"Average Loss: {np.mean(lossList):.4f}")
    print(f"Average MSE: {np.mean(mseList):.4f}")
    print(f"Average Accuracy: {np.mean(accuracyList):.4f}\n")
        # Σχεδίαση των γραφημάτων σύγκλισης
    plt.figure(figsize=(12, 5))

    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), avg_loss_per_epoch, marker='o', linestyle='-', color='r')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"Loss per Epoch (Learning Rate: {h[i]}, Momentum: {m[i]})")
    plt.grid()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), avg_acc_per_epoch, marker='o', linestyle='-', color='b')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy per Epoch (Learning Rate: {h[i]}, Momentum: {m[i]})")
    plt.grid()

    plt.show()
